## pegar dados de usuário

In [198]:
import os
from dotenv import load_dotenv

load_dotenv()

MAIL = os.getenv('UNIVESP_MAIL')
KEY = os.getenv('UNIVESP_KEY')
MAIL, KEY

('1820642@aluno.univesp.br', 'reqdsavxz01')

## importar bibliotecas

In [199]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.remote.webdriver import WebDriver
from selenium.webdriver.remote.webelement import WebElement 


In [200]:
import time
import json
import re
from typing import List, Dict

## Definições globais

In [201]:
ava_website = "https://ava.univesp.br"

semana = 1

In [212]:
class selenium_mergulhar_iframes_univesp:
  def entrar_iframes(self):
    driver = self._driver
    driver.switch_to.default_content()

    for iframe_selector in self._iframes_list:
      try:
        iframe = driver.find_element_by_css_selector(iframe_selector)
        driver.switch_to.frame(iframe)
      except:
        print('iframe $("%s") não encontrado. Continuando ainda assim...'%iframe_selector)
  def sair_iframes(self):
    driver.switch_to.default_content()

  def __enter__(self):
    self.entrar_iframes()
  def __exit__(self, exp_type, exp_val, exp_trace):
    self.sair_iframes()
  def __init__(self, driver:WebDriver, iframes_list:List[str]=['iframe']):
    self._driver = driver
    self._iframes_list = iframes_list


## Acessos gerais ao AVA

In [203]:
# função que acessa AVA com Selenium
# (deve ser fechado manualmente depois 
# por razões de multiplas célula no notebook)
def acessar_ava(driver:WebDriver=None)->WebDriver:
  if driver == None: driver = webdriver.Chrome()
  
  # abrir ava da univesp
  driver.get(ava_website)
  driver.execute_script('document.querySelector("#login-sou").click()')

  # fazer login
  # email
  driver.find_element_by_css_selector("input#username").send_keys([MAIL, Keys.ENTER])
  # senha
  driver.find_element_by_css_selector("input#password").send_keys([KEY, Keys.ENTER])

  time.sleep(5)

  return driver

In [204]:
def pegar_lista_de_materias(driver:WebDriver)->Dict[str,WebElement]:
  # acessar home das materias
  driver.get('https://ava.univesp.br/ultra/course')
  time.sleep(5)

  cartoes_materias = driver.find_elements_by_css_selector(".default-group.term-_67_1  .course-element-card")
  driver.execute_script('arguments[0].scrollIntoView({behavior:"smooth", block: "center"});', cartoes_materias[-1])
  time.sleep(5)

  # pegar lista das materias
  return {
    m.find_element_by_css_selector("h4").text.replace(" - Turma 001", '')
    :
    m
    for m in cartoes_materias
  }

## Pegar informações das matérias do bimestre

In [213]:
def pegar_info_materias(driver:WebDriver, materias:Dict[str,WebElement])->Dict[str,Dict]:
  '''
  montar JSON com info bimestral das matérias:
    titulo: {
      link
      plano de aula
      carga horaria
    }
  '''

  info_materias = {}

  # para cada materia
  for titulo, materia in materias.items():

    # abre materia
    driver.execute_script(r'arguments[0].scrollIntoView({behavior:"smooth", block: "center"});', materia)
    time.sleep(1) # espera 2 segundos para carregar
    materia.click()
    time.sleep(2) # espera 2 segundos para carregar

    # pega link da materia
    link = driver.current_url

    # entra no conteudo da univesp dentro dos iframes
    with selenium_mergulhar_iframes_univesp(driver, ['iframe']*2):
      # pega link do plano de aula
      plano_de_aula = driver.find_element_by_css_selector("a.btn.btn-syllabus").get_attribute('href')

      # pega carga horaria
      carga_horaria = driver.find_element_by_css_selector(".planejamento-estudo strong").text[:-6]

    # fecha matéria, retornando pra home
    driver.find_element_by_css_selector("button.bb-close").click()

    # guardar dados
    info_materias[titulo] = {
      "titulo": titulo,
      "link": link,
      "plano_de_aula": plano_de_aula,
      "carga_horaria": carga_horaria,
    }

  return info_materias

In [215]:
def pegar_links_semanas(driver:WebDriver, materias:Dict[str,WebElement], semanas:List[int]=None )->Dict[str, Dict[str,str]]:
  '''
  montar JSON com info semanal das matérias:
    titulo: {
      semana#: 'http...'
    }
  '''

  if not semanas: semanas = list(range(1,8))
  saida = {}

  for titulo, materia in materias.items():
    driver.execute_script(r'arguments[0].scrollIntoView({behavior:"smooth", block: "center"});', materia)

    time.sleep(1) # espera 2 segundos para carregar
    materia.click()
    time.sleep(2) # espera 2 segundos para carregar
  
    # entrar no iframe do conteudo da pagina
    with selenium_mergulhar_iframes_univesp(driver, ['iframe']):
      # pegar links das semanas
      links_menu = driver.find_elements_by_css_selector("#courseMenuPalette_contents .subhead ~ li a")
      text = lambda x: driver.execute_script('return arguments[0].innerText', x).lower()
      # links_semanas = [link.get_attribute('href') for link in links_menu if text(link)[:6]=='Semana' and text(link)[:-1] in semanas]
      links_semanas = {
        text(link).replace(" ", "_") :
        link.get_attribute('href') for link in links_menu if text(link) in ['semana %s'%n for n in semanas]  
      }

      saida[titulo] = links_semanas
     
    # fecha matéria, retornando pra home
    driver.find_element_by_css_selector("button.bb-close").click()
    
  return saida


## Pegar aulas, e exercicios

In [208]:
def pegar_info_aulas(link_semana:str, nome_semana:str='', driver:WebDriver=None ):
  '''
    pegar estrutura JSON de video aulas de uma semana de uma materia:
    {
      "titulo": "titulo do video",
      "numero": 0,
      "url": "youtube...",
      "thumb": "...jpg",
    }
  '''

  # se não informado um driver, cria um de Chrome
  localDriver = False
  if driver == None:
    driver = acessar_ava()
    localDriver = True

  saida = []

  # acessar home das materias
  driver.get(link_semana)
  time.sleep(3)

  # itens de cada tipo de conteúdo
  itens_pagina = driver.find_elements_by_css_selector('.liItem')
  itens_video_aulas = [i for i in itens_pagina if i.text.lower().startswith('videoaula')]
  itens_quiz = [i for i in itens_pagina if i.text.lower().startswith('quiz')]
  itens_exercicios_apoio = [i for i in itens_pagina if re.search("Exer.+Apoio", i.text, flags=re.I)]
  itens_atividades = [i for i in itens_pagina if re.search("avalia", i.text, flags=re.I)]


  # pegar contudos em si

  # pegar videoaulas
  for aula in itens_video_aulas:

    titulo_video = aula.find_element_by_css_selector('h3').text

    match_numero = re.search("videoaula \d+",titulo_video,flags=re.I)
    numero_aula = nome_semana[-1:]
    if match_numero:
      numero_aula = int(match_numero.group()[10:])


    # with selenium_mergulhar_iframes_univesp(['iframe']):
    driver.switch_to.default_content()
    iframe1 = aula.find_element_by_css_selector('iframe')
    driver.switch_to.frame(iframe1)

    # pegar url video do youtube
    iframe2 = driver.find_element_by_css_selector('iframe')
    url_aula = iframe2.get_attribute('src')

    # with selenium_mergulhar_iframes_univesp(['iframe']):
    driver.switch_to.frame(iframe2)

    # pegar thumbnail video youtube
    thumb_aula = re.search(
      'http.+\.\w+',
      driver.find_element_by_css_selector('.ytp-cued-thumbnail-overlay-image').get_attribute('style'),
      flags=re.I
    ).group()

    infos_aula = {
      "titulo": titulo_video,
      "numero": numero_aula,
      "url": url_aula,
      "thumb": thumb_aula,
    }

    # guardar dados da aula na saida
    driver.switch_to.default_content()
    saida.append(infos_aula)

  # TODO: pegar quizes
  # TODO: pegar atividades
  # TODO: pegar exercícios de apoio

  if localDriver:
    driver.quit()

  return saida


## Execução e montagem de JSON no squema proposto

----

In [217]:
driver = acessar_ava()
materias_atuais = pegar_lista_de_materias(driver)
info_materias = pegar_info_materias(driver, materias_atuais)
links_semanas = pegar_links_semanas(driver, materias_atuais)

In [225]:
for materia in links_semanas:
  
  if 'links_semanas' not in info_materias[materia].keys():
    info_materias[materia]['links_semanas'] = {}
  info_materias[materia]['links_semanas'].update(links_semanas[materia])

print(json.dumps(info_materias, ensure_ascii=False ,indent=4))

{
    "Engenharia de Software – EES001": {
        "titulo": "Engenharia de Software – EES001",
        "link": "https://ava.univesp.br/ultra/courses/_4646_1/cl/outline",
        "plano_de_aula": "https://ava.univesp.br/bbcswebdav/xid-3651390_1",
        "carga_horaria": "80",
        "links_semanas": {
            "semana_1": "https://ava.univesp.br/webapps/blackboard/content/listContent.jsp?course_id=_4646_1&content_id=_607081_1&mode=reset",
            "semana_2": "https://ava.univesp.br/webapps/blackboard/content/listContent.jsp?course_id=_4646_1&content_id=_607082_1&mode=reset"
        }
    },
    "Introdução aos Sistemas de Comunicação – EEL001": {
        "titulo": "Introdução aos Sistemas de Comunicação – EEL001",
        "link": "https://ava.univesp.br/ultra/courses/_4640_1/cl/outline",
        "plano_de_aula": "https://ava.univesp.br/bbcswebdav/pid-606257-dt-content-rid-3904963_1/courses/EEL001-2021S2B1C2-T001/EEL001/Slides/Semana%200%20-%20Plano%20de%20Ensino%20-%20Introdu%

In [227]:
driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="f6d9f81ae124ca70bf1b566c9b2c36ca")>

In [228]:
for materia in info_materias.values():
  materia['videoaulas'] = {}
  for semana, link in materia['links_semanas'].items():
    materia['videoaulas'][semana] = pegar_info_aulas(link, semana, driver=driver)

print(json.dumps(info_materias, ensure_ascii=False, indent=4))

{
    "Engenharia de Software – EES001": {
        "titulo": "Engenharia de Software – EES001",
        "link": "https://ava.univesp.br/ultra/courses/_4646_1/cl/outline",
        "plano_de_aula": "https://ava.univesp.br/bbcswebdav/xid-3651390_1",
        "carga_horaria": "80",
        "links_semanas": {
            "semana_1": "https://ava.univesp.br/webapps/blackboard/content/listContent.jsp?course_id=_4646_1&content_id=_607081_1&mode=reset",
            "semana_2": "https://ava.univesp.br/webapps/blackboard/content/listContent.jsp?course_id=_4646_1&content_id=_607082_1&mode=reset"
        },
        "videoaulas": {
            "semana_1": [
                {
                    "titulo": "Videoaula 1 - Modelos de processo de software e atividades de software",
                    "numero": 1,
                    "url": "https://www.youtube.com/embed/kO1PSkzTsYc",
                    "thumb": "https://i.ytimg.com/vi_webp/kO1PSkzTsYc/default.webp"
                },
                {
 

In [237]:
# salvar JSON para arquivo

# reformatar Dict para {["titulo": "materia"...]} ao inves de {"materia": {...}}
saida = [{**{"titulo": i},**info_materias[i]} for i in info_materias.keys()]


with open('info-materias.json', 'w') as f:
  json.dump(saida, f)

## Testes

----

In [ ]:
#gerarErro() # parar execução antes de fechar navegador

----

## Encerra

In [ ]:
driver.close()
driver.quit()